### Importing required libraries

In [ ]:
from src.utils import heart_slice, plot_function, plot_vector_field
from src.models.dynamics_models import BidomainModel
from src.models.cell_models import *
import ufl

### Defining a domain, cell model and the whole model and visualizing initial data

In [ ]:
domain = heart_slice()
cell_model = Noble(domain=domain)
x = ufl.SpatialCoordinate(domain)


class Model(BidomainModel):
    def initial_V_m(self):
        def value(x): 
            return -84 + 24*np.exp(-(x[0]-0)** 2 + -(x[1]-2.7) ** 2)
        self.V_m_n.interpolate(value)

    def conductivity(
        self,
        longitudinal_fibres=[
            x[1] / (x[0] ** 2 + x[1] ** 2),
            -x[0] / (x[0] ** 2 + x[1] ** 2),
        ],
        transversal_fibres=[0, 0],
    ):
        return super().conductivity(longitudinal_fibres, transversal_fibres)

    def ischemia(self):
        return (
            lambda x: (x[0] - 2) ** 2 / 10 + (x[1] - 1.2) ** 2 < 0.4**2,
            self.M_i / 10,
            self.M_e / 10,
        )


model = Model(domain, cell_model)

In [ ]:
model.plot_initial_V_m(show_mesh=False)

In [ ]:
model.plot_ischemia(show_mesh=False)

In [ ]:
x_c, y_c = 0, 0.8
plot_vector_field(
    domain,
    lambda x: [
        (x[1] - y_c) / ((x[0] - x_c) ** 2 + (x[1] - y_c) ** 2),
        -(x[0] - x_c) / ((x[0] - x_c) ** 2 + (x[1] - y_c) ** 2),
        0,
    ],
    tolerance=0.03,
    factor=1,
)

### Solving equations with given parameters

In [ ]:
model.solve(
    T=1000,
    steps=4000,
    save_to="demo.mp4",
    signal_point=[-2.0, 1.0, 0.0],
)

### Plotting a final plot of transmembrane potential

In [ ]:
plot_function(model.V_m_n, "V_m")

### Plotting a transmembrane potential in a given signal point

In [ ]:
model.plot_signal()

### Plotting a transmembrane potential solution of a cell model

In [ ]:
cell_model.visualize(1000, -84)